<a href="https://www.kaggle.com/code/juzykaggle/icr-catboost-with-preprediction-gamma?scriptVersionId=139406765" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


In [7]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from catboost import CatBoostClassifier

In [16]:
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
greek = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')

In [17]:
for i in train.drop(['EJ', 'Id'], axis=1).columns:
    train[i] = train[i].fillna(train[i].median())

In [18]:
train['EJ'] = train['EJ'].fillna(train['EJ'].mode())
train['Gamma'] = greek.Gamma.fillna(greek.Gamma.mode())

In [19]:
val, train = train[580:], train[:580]

In [20]:
X = train.drop(['Class', 'Id', 'Gamma'], axis=1)
y = train.Gamma

In [21]:
cat_feat= ['EJ']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [22]:
clf = CatBoostClassifier(cat_features=cat_feat, 
                         iterations=1300, l2_leaf_reg=7, learning_rate=0.03, depth=6).fit(X_train, y_train,
                                                                  eval_set=(X_test, y_test), plot=True, verbose=1)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 2.0533683	test: 2.0536001	best: 2.0536001 (0)	total: 68.1ms	remaining: 1m 28s
1:	learn: 2.0242393	test: 2.0256131	best: 2.0256131 (1)	total: 105ms	remaining: 1m 8s
2:	learn: 1.9913084	test: 1.9928192	best: 1.9928192 (2)	total: 142ms	remaining: 1m 1s
3:	learn: 1.9314680	test: 1.9349650	best: 1.9349650 (3)	total: 177ms	remaining: 57.4s
4:	learn: 1.9045636	test: 1.9074189	best: 1.9074189 (4)	total: 213ms	remaining: 55.2s
5:	learn: 1.8791189	test: 1.8814967	best: 1.8814967 (5)	total: 248ms	remaining: 53.6s
6:	learn: 1.8592578	test: 1.8622207	best: 1.8622207 (6)	total: 285ms	remaining: 52.6s
7:	learn: 1.8300585	test: 1.8336353	best: 1.8336353 (7)	total: 324ms	remaining: 52.3s
8:	learn: 1.7877939	test: 1.7907750	best: 1.7907750 (8)	total: 360ms	remaining: 51.6s
9:	learn: 1.7608005	test: 1.7618768	best: 1.7618768 (9)	total: 396ms	remaining: 51s
10:	learn: 1.7363020	test: 1.7395587	best: 1.7395587 (10)	total: 432ms	remaining: 50.6s
11:	learn: 1.7009252	test: 1.7036586	best: 1.7036586

In [24]:
clf.score(X_test, y_test)

0.8072916666666666

In [25]:
clf.score(val.drop(['Class', 'Id', 'Gamma'], axis=1), val.Gamma)

0.918918918918919

In [26]:
test['Gamma'] = clf.predict(test.drop(['Id'], axis=1))

In [27]:
test

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Gamma
0,00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,M
1,010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,M
2,02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,M
3,040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,M
4,046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,M


In [28]:
X_final = train.drop(['Class', 'Id'], axis=1)
y_final = train.Class
X_final_train, X_final_test, y_final_train, y_final_test = train_test_split(X_final, y_final, test_size=0.1, random_state=42)

In [30]:
cat_feat_final = ['EJ', 'Gamma']

In [31]:
clf_final = CatBoostClassifier(cat_features=cat_feat_final, 
                               iterations=100, l2_leaf_reg=4, 
                               learning_rate=0.07, depth=4).fit(X_final_train, y_final_train,
                                                                eval_set=(X_final_test, y_final_test), plot=True, verbose=1)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6298184	test: 0.6362662	best: 0.6362662 (0)	total: 6.81ms	remaining: 674ms
1:	learn: 0.4885285	test: 0.4935542	best: 0.4935542 (1)	total: 11.9ms	remaining: 582ms
2:	learn: 0.3804213	test: 0.3839749	best: 0.3839749 (2)	total: 15.7ms	remaining: 508ms
3:	learn: 0.3049788	test: 0.3068041	best: 0.3068041 (3)	total: 19.6ms	remaining: 470ms
4:	learn: 0.2437875	test: 0.2440005	best: 0.2440005 (4)	total: 23.3ms	remaining: 443ms
5:	learn: 0.2004980	test: 0.1996543	best: 0.1996543 (5)	total: 27.3ms	remaining: 428ms
6:	learn: 0.1605463	test: 0.1585882	best: 0.1585882 (6)	total: 31.1ms	remaining: 413ms
7:	learn: 0.1323804	test: 0.1304719	best: 0.1304719 (7)	total: 35ms	remaining: 402ms
8:	learn: 0.1074366	test: 0.1070135	best: 0.1070135 (8)	total: 38.6ms	remaining: 390ms
9:	learn: 0.0895401	test: 0.0886730	best: 0.0886730 (9)	total: 42.3ms	remaining: 381ms
10:	learn: 0.0719620	test: 0.0720650	best: 0.0720650 (10)	total: 46.1ms	remaining: 373ms
11:	learn: 0.0613953	test: 0.0614856	best: 

In [32]:
clf_final.score(X_final_test, y_final_test)

1.0

In [33]:
clf_final.score(val.drop(['Class', 'Id'], axis=1), val.Class)

1.0

In [34]:
y_pred = clf_final.predict_proba(test.drop(['Id'], axis=1))

In [35]:
submission = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')

In [36]:
submission[['class_0', 'class_1']] = y_pred

In [37]:
submission

,Id,class_0,class_1
0,00eed32682bb,0.994344,0.005656
1,010ebe33f668,0.994344,0.005656
2,02fa521e1838,0.994344,0.005656
3,040e15f562a2,0.994344,0.005656
4,046e85c7cc7f,0.994344,0.005656


In [38]:
submission.to_csv('submission.csv', index=False)